# Eval Driven Development with MLflow & LangChain

This notebook demonstrates how to perform **Evaluation Driven Development (EDD)** for GenAI applications using **MLflow 3.0+** and **LangChain**.

We will cover two main scenarios:
1.  **RAG Evaluation**: Using built-in LLM judges (`RetrievalGroundedness`, `RetrievalRelevance`, etc.) to evaluate a retrieval system.
2.  **Agent Evaluation**: Using custom scorers to inspect execution traces and validate tool usage trajectories.

### Prerequisites
Ensure you have set your `OPENAI_API_KEY` in the environment or a `.env` file.

In [1]:
# Install dependencies if running in Colab or a fresh environment
#%pip install -q "mlflow>=2.14" langgraph langchain langchain-openai langchain-community langchain-text-splitters faiss-cpu pandas openai python-dotenv bs4

In [2]:
import mlflow
print(f"MLflow version: {mlflow.__version__}")

MLflow version: 3.6.0


In [3]:
import os
import mlflow
import pandas as pd
from dotenv import load_dotenv

# Load API Key
load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("Enter your OpenAI API Key: ")

# Set a specific experiment for this notebook
mlflow.set_experiment("GenAI_Eval_Demo")

/Users/pedro.azevedo/dspt-mlflow/.venv/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)


<Experiment: artifact_location='file:///Users/pedro.azevedo/dspt-mlflow/mlruns/413835162552422093', creation_time=1763905836235, experiment_id='413835162552422093', last_update_time=1763905836235, lifecycle_stage='active', name='GenAI_Eval_Demo', tags={}>

## Setup RAG Agent

In [23]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

docs = [
        Document(page_content="Paul Graham grew up writing short stories and programming on an IBM 1401."),
        Document(page_content="After Y Combinator, Paul Graham spent time painting and working on Lisp."),
        Document(page_content="Paul Graham co-founded Y Combinator, one of the most successful startup accelerators."),
        Document(page_content="Paul Graham wrote influential essays about startups, programming, and technology."),
        Document(page_content="Before Y Combinator, Paul Graham worked on Viaweb, which was later sold to Yahoo."),
    ]
# Create Vector Store & Retriever
vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [24]:
from langchain.tools import tool
import os
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent

model = init_chat_model("gpt-4.1")


@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vectorstore.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs




tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

## Setup Predict Functions

In [106]:
def qa_precict_fn(query: str) -> str:
    response = agent.invoke({
        "messages": [{"role": "user", "content": query}],
    })
    answer = response['messages'][-1].content
    return answer

## Part 1: RAG Evaluation

We will build a simple RAG chain that answers questions about software tools. We will then evaluate it using MLflow's **"Trace Required"** judges, which inspect the actual retrieved documents to ensure relevance and groundedness.

In [ ]:
from mlflow.genai.scorers import (
    Correctness,
    RelevanceToQuery,
    Guidelines,
)
from mlflow.entities import Feedback, SpanType, Trace
from mlflow.genai import scorer
from deepeval.metrics import TaskCompletionMetric
from deepeval.test_case import LLMTestCase, ToolCall
import json
from deepeval.metrics import ContextualRelevancyMetric


# 1. Enable Autologging
mlflow.langchain.autolog()

## Setup RAG Eval Dataset

In [108]:
# RAG evaluation dataset with questions closely related to the provided docs
# Updated to include task completion and tool trajectory expectations
rag_eval_dataset = [
    # Childhood and early interests
    {
        "inputs": {"query": "What did Paul Graham do when he was young?"},
        "expectations": {
            "expected_response": "Paul Graham grew up writing short stories and programming on an IBM 1401.",
            "expected_facts": ["writing short stories", "programming on an IBM 1401"],
            "task_completion_threshold": 0.8,
            "tool_call_trajectory": ["retrieve_context"],  # Expected tools for simple factual query
            "expected_tools": ["retrieve_context"],
        }
    },
    # Activities after Y Combinator
    {
        "inputs": {"query": "What did Paul Graham pursue after leaving Y Combinator?"},
        "expectations": {
            "expected_response": "After Y Combinator, Paul Graham spent time painting and working on Lisp.",
            "expected_facts": ["painting", "working on Lisp"],
            "task_completion_threshold": 0.8,
            "tool_call_trajectory": ["retrieve_context"],
            "expected_tools": ["retrieve_context"],
        }
    },
    # Startup accelerator founding
    {
        "inputs": {"query": "Who co-founded Y Combinator and what is its significance?"},
        "expectations": {
            "expected_response": "Paul Graham co-founded Y Combinator, one of the most successful startup accelerators.",
            "expected_facts": ["Paul Graham co-founded Y Combinator", "successful startup accelerator"],
            "task_completion_threshold": 0.7,
            "tool_call_trajectory": ["retrieve_context"],
            "expected_tools": ["retrieve_context"],
        }
    },
    # Essays and writing
    {
        "inputs": {"query": "What topics did Paul Graham write essays about?"},
        "expectations": {
            "expected_response": "Paul Graham wrote influential essays about startups, programming, and technology.",
            "expected_facts": ["startups", "programming", "technology"],
            "task_completion_threshold": 0.8,
            "tool_call_trajectory": ["retrieve_context"],
            "expected_tools": ["retrieve_context"],
        }
    },
    # Complex multi-step query
    {
        "inputs": {"query": "Compare Paul Graham's work before and after Y Combinator, including his early career and later pursuits"},
        "expectations": {
            "expected_response": "Before Y Combinator, Paul Graham worked on Viaweb which was sold to Yahoo, and grew up programming on IBM 1401. After Y Combinator, he spent time painting and working on Lisp while writing influential essays.",
            "expected_facts": ["Viaweb", "sold to Yahoo", "painting", "working on Lisp", "writing essays"],
            "task_completion_threshold": 0.6,  # Lower threshold for complex task
            "tool_call_trajectory": ["retrieve_context", "retrieve_context"],  # May involve multiple retrieval calls
            "expected_tools": ["retrieve_context"],
        }
    },
    # Early career and Viaweb
    {
        "inputs": {"query": "What company did Paul Graham work on before Y Combinator and what happened to it?"},
        "expectations": {
            "expected_response": "Before Y Combinator, Paul Graham worked on Viaweb, which was later sold to Yahoo.",
            "expected_facts": ["Viaweb", "sold to Yahoo"],
            "task_completion_threshold": 0.8,
            "tool_call_trajectory": ["retrieve_context","retrieve_context"],
            "expected_tools": ["retrieve_context"],
        }
    }
]

### Helper Functions to Process Traces

In [ ]:


def extract_source_nodes(json_input):
    """
    Parses a JSON string containing a message history and extracts source nodes
    from tool artifacts.
    """
    try:
        parsed_data = json.loads(json_input)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return []

    # Handle the structure: {"messages": [...]}
    messages = parsed_data.get("messages", []) if isinstance(parsed_data, dict) else []
    
    source_nodes = []
    
    for message in messages:
        # We are looking for messages where type is 'tool' and an 'artifact' list exists
        if message.get("type") == "tool" and "artifact" in message:
            artifacts = message["artifact"]
            
            # Ensure artifact is a list before extending our results
            if isinstance(artifacts, list):
                source_nodes.extend(artifacts)
                
    return source_nodes

def extract_final_response(json_input):
    """
    Parses a JSON string and extracts the content of the final AI response.
    """
    try:
        parsed_data = json.loads(json_input)
    except json.JSONDecodeError:
        return None

    messages = parsed_data.get("messages", []) if isinstance(parsed_data, dict) else []
    
    # Iterate backwards to find the most recent AI message with content
    for message in reversed(messages):
        if message.get("type") == "ai" and message.get("content"):
            return message["content"]
            
    return None


## Setup Scorers and Evaluators

In [114]:


@scorer(name="Task Completeness")
def task_completion_with_deepeval(trace: Trace, inputs: dict, outputs: str, expectations: dict) -> Feedback:
    """
    Custom scorer that uses DeepEval's TaskCompletionMetric to evaluate task completion
    based on trace analysis and tool calls
    """

    try:
        # Extract tool call information from the trace
        tool_call_spans = trace.search_spans(span_type=SpanType.TOOL)

        # Convert MLflow trace tool calls to DeepEval ToolCall format
        tools_called = []
        for span in tool_call_spans:
            tool_call = ToolCall(
                name=span.name,
                description=span.attributes.get("description", f"Tool call for {span.name}"),
                input_parameters=span.inputs or {},
                output=span.outputs or []
            )
            tools_called.append(tool_call)

        # Extract the actual response text from the complex output structure
        if isinstance(outputs, dict):
            # Handle complex response structure
            if 'response' in outputs and 'blocks' in outputs['response']:
                actual_output = outputs['response']['blocks'][0]['text']
            elif 'response' in outputs and isinstance(outputs['response'], str):
                actual_output = outputs['response']
            else:
                actual_output = str(outputs)
        elif isinstance(outputs, str):
            actual_output = outputs
        else:
            actual_output = str(outputs)

        # Create DeepEval test case
        test_case = LLMTestCase(
            input=inputs.get("query", ""),
            actual_output=actual_output,
            tools_called=tools_called
        )

        # Initialize TaskCompletionMetric
        threshold = expectations.get("task_completion_threshold", 0.7)
        metric = TaskCompletionMetric(
            threshold=threshold,
            model="gpt-4o",  # Use consistent model
            include_reason=True
        )

        # Run the metric evaluation
        metric.measure(test_case)

        # Extract results
        score = metric.score
        reason = metric.reason

        return Feedback(
            value=score,
            rationale=f"Task completion score: {score:.2f} (threshold: {threshold}). Tools used: {len(tools_called)}. {reason}",
        )

    except Exception as e:
        return Feedback(
            value=0.0,
            rationale=f"Error evaluating task completion: {str(e)}",
            error=e
        )


@scorer(name="Tool Trajectory")
def tool_call_trajectory_analysis(trace: Trace, expectations: dict) -> Feedback:
    """
    Analyze the tool call trajectory against expected sequence
    """
    try:
        # Search for tool call spans in the trace
        tool_call_spans = trace.search_spans(span_type=SpanType.TOOL)

        # Extract actual trajectory
        actual_trajectory = [span.name for span in tool_call_spans]
        expected_trajectory = expectations.get("tool_call_trajectory", [])

        # Calculate trajectory match
        trajectory_match = actual_trajectory == expected_trajectory

        # Calculate partial match score
        if not expected_trajectory:
            partial_score = 1.0 if actual_trajectory else 0.0
        else:
            # Calculate sequence similarity
            min_len = min(len(actual_trajectory), len(expected_trajectory))
            max_len = max(len(actual_trajectory), len(expected_trajectory))
            if max_len == 0:
                partial_score = 1.0
            else:
                matches = sum(1 for i in range(min_len)
                             if i < len(actual_trajectory) and i < len(expected_trajectory)
                             and actual_trajectory[i] == expected_trajectory[i])
                partial_score = matches / max_len

        return Feedback(
            value=partial_score,
            rationale=(
                f"Tool trajectory {'matches' if trajectory_match else 'differs from'} expectations. "
                f"Expected: {expected_trajectory}. Actual: {actual_trajectory}. "
                f"Match score: {partial_score:.2f}"
            )
        )

    except Exception as e:
        return Feedback(
            value=0.0,
            rationale=f"Error analyzing tool trajectory: {str(e)}",
            error=e
        )
    



def _extract_deepeval_components(trace : Trace):
    """Extract input, output, and context from trace data"""
    request = str(trace.data.request)
    response = str(trace.data.response)

    # extract source nodes if they exist
    # Extract Source Nodes
    outputs = extract_source_nodes(response)
    retrieval_context = [node['page_content'] for node in outputs]
    
    
    actual_output = extract_final_response(response)

    return {
        'input': request,
        'actual_output': actual_output,
        'retrieval_context': retrieval_context
    }


@scorer(name="Context Relevancy")
def contextual_relevancy(trace: Trace):
    """Evaluate RAG traces using DeepEval metrics"""
    
    data = _extract_deepeval_components(trace)

    # Define DeepEval metrics   
    

    # Create test cases
    test_case= LLMTestCase(
            input=data['input'],
            actual_output=data['actual_output'],
            retrieval_context=data['retrieval_context']
        )
    
    relevancy_metric = ContextualRelevancyMetric(threshold=0.7, model="gpt-4o-mini",
                                        include_reason=True)        
        
    # Run the metric evaluation
    relevancy_metric.measure(test_case)

    # Extract results
    score = relevancy_metric.score
    reason = relevancy_metric.reason

    return Feedback(
        value=score,
        rationale=reason,
    )

# Business-specific guidelines
business_guidelines = Guidelines(
    name="toxicity_clear",
    guidelines="""
    The response should avoid toxic language and adhere to community guidelines.
    """,
)

# Configure RAG-specific scorers
retrieval_scorers = [
    Correctness(
       # model="litellm_proxy:/amazon.nova-micro-v1:0",
    ),
    RelevanceToQuery(
        name="AnswerRelevance"
        #model="litellm_proxy:/amazon.nova-micro-v1:0",
    ),
    contextual_relevancy,
]

agent_scorers = [
    task_completion_with_deepeval,
    tool_call_trajectory_analysis,
]

all_scorers= retrieval_scorers + agent_scorers +[
    business_guidelines, 
]

In [125]:
import mlflow

with mlflow.start_run(run_name="Simple Langgraph Agent"):

    eval_results = mlflow.genai.evaluate(
        data=rag_eval_dataset,
        predict_fn=qa_precict_fn,
        scorers=all_scorers,
    )

2025/11/23 18:06:47 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.
2025/11/23 18:06:47 WARNING mlflow.tracing.fluent: Failed to start span LangGraph: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
Evaluating:   0%|          | 0/6 [Elapsed: 00:00, Remaining: ?] 

/Users/pedro.azevedo/dspt-mlflow/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/pedro.azevedo/dspt-mlflow/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/pedro.azevedo/dspt-mlflow/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/pedro.azevedo/dspt-mlflow/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

Evaluating:  83%|████████▎ | 5/6 [Elapsed: 00:09, Remaining: 00:01] 

/Users/pedro.azevedo/dspt-mlflow/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/pedro.azevedo/dspt-mlflow/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Evaluating: 100%|██████████| 6/6 [Elapsed: 00:12, Remaining: 00:00] 



✨ Evaluation completed.

Metrics and evaluation results are logged to the MLflow run:
  Run name: Simple Langgraph Agent
  Run ID: 4e35361c32984e39b6cb4020dd747698

To view the detailed evaluation results with sample-wise scores,
open the Traces tab in the Run page in the MLflow UI.



In [123]:
eval_results.metrics

{'Tool Trajectory/mean': np.float64(0.9166666666666666),
 'toxicity_clear/mean': np.float64(1.0),
 'AnswerRelevance/mean': np.float64(1.0),
 'correctness/mean': np.float64(0.8333333333333334),
 'Task Completeness/mean': np.float64(0.8916666666666667),
 'Context Relevancy/mean': np.float64(0.4583333333333333)}

## Part 2: Agent Evaluation

Now we evaluate a "DevOps Agent" that has tools to check server status and restart servers. We want to ensure:
1.  **Trajectory**: It checks status *before* restarting (SOP compliance).
2.  **Safety**: It never attempts to restart 'prod' without safety checks (simulated here by a hard fail if it tries).